#Image Classification - Task 2

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
%tensorflow_version 2.x
import tensorflow as tf
import os
from os import path


print(tf.__version__)

%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.vision import *
from fastai.vision.transform import *
from fastai.metrics import error_rate, accuracy

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="torch.nn.functional")

import pandas as pd

TensorFlow 2.x selected.
2.0.0


In [0]:
bs = 128
np.random.seed(5432)

root_path = F'/content/gdrive/My Drive/SkyHack 2019 - Team'
img_path = path.join(root_path, 'data', 'Task 2 data')

In [0]:
def save_data(data, path, name):
  with open(os.path.join(path, name+".pickle"), 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)
  
def read_data(path, name):
  with open(os.path.join(path, name), 'rb') as handle:
    data = pickle.load(handle)
  return data

**Model 2**: Acc: 90%, resnet34, standard transoforms and normlized.

In [0]:
tfms = get_transforms()
data = ImageDataBunch.from_folder(img_path, train='raw_data', valid_pct=0.2,size=500, test='validation', ds_tfms=tfms).normalize()

In [0]:
# Walidacja
print(len(data.train_ds))
print(len(data.valid_ds))
print(len(data.test_ds))

print(data.classes)

1787
446
552
['bathroom', 'bedroom', 'dinning_room', 'house', 'kitchen', 'living_room']


In [0]:
learn = cnn_learner(data, models.resnet34, metrics=accuracy)
learn.fit_one_cycle(15)
learn.save('task_2_model_2')
save_data(data, img_path, "task_2_model_2")

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/checkpoints/resnet34-333f7ec4.pth
100%|██████████| 83.3M/83.3M [00:03<00:00, 28.7MB/s]


epoch,train_loss,valid_loss,accuracy,time
0,1.953569,0.849875,0.665919,03:57
1,1.267318,0.537849,0.813901,02:39
2,0.892364,0.486211,0.845291,02:41
3,0.672833,0.489774,0.834081,02:39
4,0.543153,0.470916,0.845291,02:39
5,0.436073,0.476857,0.852018,02:40
6,0.360885,0.455804,0.869955,02:39
7,0.306880,0.448592,0.858744,02:48
8,0.258736,0.430063,0.854260,02:40
9,0.217477,0.420294,0.867713,02:42


**Model 3**: 

In [0]:
tfms = get_transforms(max_lighting=0.2, p_lighting=0.5)
data = ImageDataBunch.from_folder(img_path, train='raw_data', valid_pct=0.2, size = 500,test='validation', ds_tfms=tfms).normalize()

In [0]:
# Walidacja
print(len(data.train_ds))
print(len(data.valid_ds))
print(len(data.test_ds))

1787
446
552


In [0]:
learn = cnn_learner(data, models.resnet34, metrics=accuracy)
learn.fit_one_cycle(15)
learn.save('model-3')

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/checkpoints/resnet34-333f7ec4.pth
100%|██████████| 83.3M/83.3M [00:01<00:00, 73.7MB/s]


epoch,train_loss,valid_loss,accuracy,time
0,2.273694,0.839344,0.683856,04:10
1,1.367212,0.553464,0.807175,02:52
2,0.956279,0.538377,0.825112,02:45
3,0.711829,0.510387,0.840807,02:43
4,0.555237,0.506783,0.843049,02:49
5,0.451582,0.466289,0.840807,02:47
6,0.373156,0.459488,0.843049,02:48
7,0.316182,0.430398,0.852018,02:49
8,0.268798,0.444291,0.867713,02:48
9,0.228411,0.424031,0.872197,02:44


**Model 4**

In [0]:
tfms = get_transforms()
data = ImageDataBunch.from_folder(path.join(root_path, "Bart"), train='raw_data', valid_pct=0.2, size = 500,test='validation', ds_tfms=tfms).normalize()

In [0]:
# Walidacja
print(len(data.train_ds))
print(len(data.valid_ds))
print(len(data.test_ds))

3697
924
552


In [0]:
learn = cnn_learner(data, models.resnet34, metrics=accuracy)
learn.fit_one_cycle(15)
learn.save('model-4')

epoch,train_loss,valid_loss,accuracy,time
0,1.654639,0.700694,0.744589,07:01
1,1.048968,0.513515,0.820346,03:57
2,0.765992,0.463869,0.843074,03:50
3,0.632773,0.469675,0.843074,03:54
4,0.548793,0.416822,0.848485,03:54
5,0.482749,0.410672,0.858225,03:56
6,0.421388,0.395217,0.862554,03:56
7,0.365151,0.387331,0.862554,03:57
8,0.331875,0.376755,0.862554,04:01
9,0.294905,0.365763,0.874459,03:59


In [0]:
import pickle

with open('dane_model_4.pickle', 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

dane_model_4.pickle  gdrive  sample_data


In [0]:
learn.find_lr()
learn.recorder.plot()

low_lr = 0
high_lr = 0

In [0]:
learn.unfreez()
learn.fit_one_cycle(2, max_lr=(low_lr,high_lr))

### Prediction

In [0]:
learn = cnn_learner(data, models.resnet34, metrics=error_rate)
learn = learn.load("model-2")

In [0]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_top_losses(9, heatmap=False)
#interp.plot_confusion_matrix()

In [0]:
img = data.test_ds[20][0]
img

In [0]:
learn.predict(img)